In [16]:
import numpy as np
import pandas as pd
import re
import ast
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('../data/raw/appartments.csv')

In [3]:
df.drop(22,inplace=True)
df.head(3)

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."


In [4]:
def extract_list(s):
    return re.findall(r"'(.*?)'", s)

In [5]:
df['TopFacilities'][0]

"['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"

In [6]:
# Converting the all facilities in list
df['TopFacilities'] = df['TopFacilities'].apply(extract_list)
df['TopFacilities'][0]

['Swimming Pool',
 'Salon',
 'Restaurant',
 'Spa',
 'Cafeteria',
 'Sun Deck',
 '24x7 Security',
 'Club House',
 'Gated Community']

In [7]:
# Converting to string for vactorization
df['FacilitiesStr'] = df['TopFacilities'].apply(' '.join)

df['FacilitiesStr'][0]


'Swimming Pool Salon Restaurant Spa Cafeteria Sun Deck 24x7 Security Club House Gated Community'

In [45]:
type(tfidf_matrix)

scipy.sparse._csr.csr_matrix

In [8]:
# using tfidf

tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
tfidf_matrix = tfidf_vectorizer.fit_transform(df['FacilitiesStr'])
tfidf_matrix.toarray()[0]

array([0.        , 0.        , 0.        , 0.18809342, 0.18809342,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [9]:
# using cosine similarity to measure similarity between properties

cosine_sim1 = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim1.shape

(246, 246)

In [10]:
cosine_sim1

array([[1.        , 0.01095159, 0.        , ..., 0.01183329, 0.08656385,
        0.0110727 ],
       [0.01095159, 1.        , 0.01982121, ..., 0.11904241, 0.01555534,
        0.00963852],
       [0.        , 0.01982121, 1.        , ..., 0.07020502, 0.03820314,
        0.01962826],
       ...,
       [0.01183329, 0.11904241, 0.07020502, ..., 1.        , 0.09825738,
        0.03255851],
       [0.08656385, 0.01555534, 0.03820314, ..., 0.09825738, 1.        ,
        0.06257614],
       [0.0110727 , 0.00963852, 0.01962826, ..., 0.03255851, 0.06257614,
        1.        ]])

In [11]:
def recommend_properties(property_name, cosine_sim=cosine_sim1):
    # Get the index of the property that matches the name
    idx = df.index[df['PropertyName'] == property_name].tolist()[0]

    # Get the pairwise similarity scores with that property
    sim_scores = list(enumerate(cosine_sim1[idx]))

    # Sort the properties based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 5 most similar properties
    sim_scores = sim_scores[1:6]

    # Get the property indices
    property_indices = [i[0] for i in sim_scores]
    
    recommendations_df = pd.DataFrame({
        'PropertyName': df['PropertyName'].iloc[property_indices],
        'SimilarityScore': sim_scores
    })

    # Return the top 10 most similar properties
    return recommendations_df

In [12]:
recommend_properties("DLF The Arbour")

,PropertyName,SimilarityScore
64,Ace Palm Floors,"(63, 0.4529382062441955)"
217,Yashika 104,"(216, 0.4199606322926784)"
93,JMS The Nation,"(92, 0.4166584649363288)"
154,India Rashtra,"(153, 0.398954234680194)"
0,Smartworld One DXP,"(0, 0.38885046199432893)"


In [24]:
def refined_parse_modified_v2(detail_str):
    try:
        details = json.loads(detail_str.replace("'", "\""))
    except:
        return {}

    extracted = {}
    for bhk, detail in details.items():
        # Extract building type
        extracted[f'building type_{bhk}'] = detail.get('building_type')

        # Parsing area details
        area = detail.get('area', '')
        area_parts = area.split('-')
        if len(area_parts) == 1:
            try:
                value = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area low {bhk}'] = value
                extracted[f'area high {bhk}'] = value
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None
        elif len(area_parts) == 2:
            try:
                extracted[f'area low {bhk}'] = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area high {bhk}'] = float(area_parts[1].replace(',', '').replace(' sq.ft.', '').strip())
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None

        # Parsing price details
        price_range = detail.get('price-range', '')
        price_parts = price_range.split('-')
        if len(price_parts) == 2:
            try:
                extracted[f'price low {bhk}'] = float(price_parts[0].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                extracted[f'price high {bhk}'] = float(price_parts[1].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                if 'L' in price_parts[0]:
                    extracted[f'price low {bhk}'] /= 100
                if 'L' in price_parts[1]:
                    extracted[f'price high {bhk}'] /= 100
            except:
                extracted[f'price low {bhk}'] = None
                extracted[f'price high {bhk}'] = None

    return extracted

In [25]:
data_refined = []

for _, row in df.iterrows():
    features = refined_parse_modified_v2(row['PriceDetails'])
    
    # Construct a new row for the transformed dataframe
    new_row = {'PropertyName': row['PropertyName']}
    
    # Populate the new row with extracted features
    for config in ['1 BHK', '2 BHK', '3 BHK', '4 BHK', '5 BHK', '6 BHK', '1 RK', 'Land']:
        new_row[f'building type_{config}'] = features.get(f'building type_{config}')
        new_row[f'area low {config}'] = features.get(f'area low {config}')
        new_row[f'area high {config}'] = features.get(f'area high {config}')
        new_row[f'price low {config}'] = features.get(f'price low {config}')
        new_row[f'price high {config}'] = features.get(f'price high {config}')
    
    data_refined.append(new_row)

In [26]:
df_final_refined_v2 = pd.DataFrame(data_refined).set_index('PropertyName')
df_final_refined_v2

,building type_1 BHK,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,building type_2 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,...,building type_1 RK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,building type_Land,area low Land,area high Land,price low Land,price high Land
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,None,NaN,NaN,NaN,NaN,Apartment,1370.0,1370.0,2.0000,2.40,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
M3M Crown,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Adani Brahma Samsara Vilasa,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,,500.0,4329.0,2.05,41.13
Sobha City,None,NaN,NaN,NaN,NaN,Apartment,1381.0,1692.0,1.5500,3.21,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Signature Global City 93,None,NaN,NaN,NaN,NaN,Independent Floor,981.0,1118.0,0.9301,1.06,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,None,NaN,NaN,NaN,NaN,Apartment,964.0,964.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Pyramid Urban Homes 2,Apartment,335.0,398.0,23.45,0.2786,Apartment,500.0,625.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Satya The Hermitage,None,NaN,NaN,NaN,NaN,Apartment,1450.0,1450.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [27]:
df_final_refined_v2['building type_Land'] = df_final_refined_v2['building type_Land'].replace({'':'Land'})
df_final_refined_v2

,building type_1 BHK,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,building type_2 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,...,building type_1 RK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,building type_Land,area low Land,area high Land,price low Land,price high Land
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,None,NaN,NaN,NaN,NaN,Apartment,1370.0,1370.0,2.0000,2.40,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
M3M Crown,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Adani Brahma Samsara Vilasa,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,Land,500.0,4329.0,2.05,41.13
Sobha City,None,NaN,NaN,NaN,NaN,Apartment,1381.0,1692.0,1.5500,3.21,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Signature Global City 93,None,NaN,NaN,NaN,NaN,Independent Floor,981.0,1118.0,0.9301,1.06,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,None,NaN,NaN,NaN,NaN,Apartment,964.0,964.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Pyramid Urban Homes 2,Apartment,335.0,398.0,23.45,0.2786,Apartment,500.0,625.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Satya The Hermitage,None,NaN,NaN,NaN,NaN,Apartment,1450.0,1450.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [42]:
df['PriceDetails'][10]


"{'2 BHK': {'building_type': 'Independent Floor', 'area_type': 'Carpet Area', 'area': '1,055 sq.ft.', 'price-range': '₹ 1.05 - 1.5 Cr'}, '3 BHK': {'building_type': 'Independent Floor', 'area_type': 'Carpet Area', 'area': '1,325 - 1,525 sq.ft.', 'price-range': '₹ 1.35 - 1.84 Cr'}}"

In [43]:
df_final_refined_v2

,building type_1 BHK,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,building type_2 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,...,building type_1 RK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,building type_Land,area low Land,area high Land,price low Land,price high Land
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
M3M Crown,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Adani Brahma Samsara Vilasa,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Sobha City,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Signature Global City 93,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Pyramid Urban Homes 2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Satya The Hermitage,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [28]:
categorical_columns = df_final_refined_v2.select_dtypes(include=['object']).columns.tolist()
categorical_columns

['building type_1 BHK',
 'building type_2 BHK',
 'building type_3 BHK',
 'building type_4 BHK',
 'building type_5 BHK',
 'building type_6 BHK',
 'building type_1 RK',
 'building type_Land']

In [30]:
ohe_df = pd.get_dummies(df_final_refined_v2, columns=categorical_columns, drop_first=True)
ohe_df.fillna(0,inplace=True)
ohe_df

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,...,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,0.0,0.0,0.00,0.0000,1370.0,1370.0,2.0000,2.40,1850.0,2050.0,...,False,False,False,False,False,False,False,False,False,False
M3M Crown,0.0,0.0,0.00,0.0000,0.0,0.0,0.0000,0.00,1605.0,2170.0,...,False,False,False,False,False,False,False,False,False,False
Adani Brahma Samsara Vilasa,0.0,0.0,0.00,0.0000,0.0,0.0,0.0000,0.00,1800.0,3150.0,...,False,False,True,False,False,True,False,False,False,False
Sobha City,0.0,0.0,0.00,0.0000,1381.0,1692.0,1.5500,3.21,1711.0,2343.0,...,False,False,False,False,False,False,False,False,False,False
Signature Global City 93,0.0,0.0,0.00,0.0000,981.0,1118.0,0.9301,1.06,1235.0,1530.0,...,True,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,0.0,0.0,0.00,0.0000,964.0,964.0,0.0000,0.00,1127.0,1127.0,...,False,False,False,False,False,False,False,False,False,False
Pyramid Urban Homes 2,335.0,398.0,23.45,0.2786,500.0,625.0,0.0000,0.00,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
Satya The Hermitage,0.0,0.0,0.00,0.0000,1450.0,1450.0,0.0000,0.00,1991.0,1991.0,...,False,False,False,False,False,False,False,False,False,False


In [31]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
ohe_df_normalized = pd.DataFrame(scaler.fit_transform(ohe_df), columns=ohe_df.columns, index=ohe_df.index)
ohe_df_normalized

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,...,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-0.252266,-0.169584,-0.105197,-0.082332,1.223499,1.020101,-0.173712,1.158423,0.553787,0.370864,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
M3M Crown,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.293086,0.472749,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Adani Brahma Samsara Vilasa,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.500583,1.304803,...,-0.289310,-0.063888,2.683282,-0.063888,-0.171139,3.924283,-0.236208,-0.111111,-0.216353,-0.063888
Sobha City,-0.252266,-0.169584,-0.105197,-0.082332,1.240497,1.470610,-0.198425,1.680336,0.405879,0.619632,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Signature Global City 93,-0.252266,-0.169584,-0.105197,-0.082332,0.622383,0.667529,-0.232468,0.295011,-0.100626,-0.070634,...,3.456497,-0.063888,2.683282,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,-0.252266,-0.169584,-0.105197,-0.082332,0.596113,0.452068,-0.283546,-0.387986,-0.215547,-0.412795,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Pyramid Urban Homes 2,1.565039,0.704171,9.593528,0.224987,-0.120899,-0.022226,-0.283546,-0.387986,-1.414772,-1.369658,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Satya The Hermitage,-0.252266,-0.169584,-0.105197,-0.082332,1.347122,1.132029,-0.283546,-0.387986,0.703823,0.320771,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888


In [32]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim2 = cosine_similarity(ohe_df_normalized)
cosine_sim2

array([[ 1.        ,  0.13237477, -0.03199582, ...,  0.23938838,
         0.09359211,  0.3918892 ],
       [ 0.13237477,  1.        ,  0.08163866, ..., -0.28145036,
        -0.15433359, -0.13962499],
       [-0.03199582,  0.08163866,  1.        , ..., -0.23191291,
        -0.41377594, -0.32401234],
       ...,
       [ 0.23938838, -0.28145036, -0.23191291, ...,  1.        ,
         0.10863087,  0.18656375],
       [ 0.09359211, -0.15433359, -0.41377594, ...,  0.10863087,
         1.        ,  0.88701018],
       [ 0.3918892 , -0.13962499, -0.32401234, ...,  0.18656375,
         0.88701018,  1.        ]])

In [33]:
def recommend_properties_with_scores(property_name, top_n=247):
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim2[ohe_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = ohe_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

In [34]:
recommend_properties_with_scores('M3M Golf Hills')


,PropertyName,SimilarityScore
0,AIPL The Peaceful Homes,0.955462
1,Smartworld One DXP,0.954670
2,Unitech Escape,0.953092
3,M3M Capital,0.951156
4,BPTP Terra,0.943128
...,...,...
240,Golden Park,-0.522391
241,Satya Merano Greens,-0.523660
242,ROF Normanton Park,-0.525129
243,BPTP Green Oaks,-0.525286


In [35]:
def distance_to_meters(distance_str):
    try:
        if 'Km' in distance_str or 'KM' in distance_str:
            return float(distance_str.split()[0]) * 1000
        elif 'Meter' in distance_str or 'meter' in distance_str:
            return float(distance_str.split()[0])
        else:
            return None
    except:
        return None

In [65]:
import re
import logging

# Configure logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def distance_to_meters(distance_str: str) -> float:
    """
    Convert distance string to meters.
    
    Parameters:
    distance_str (str): The distance string containing 'Km', 'KM', 'Meter', or 'meter'.
    
    Returns:
    float: The distance in meters or None if conversion fails.
    """
    try:
        match = re.search(r"(\d+(\.\d+)?)\s*(Km|KM|km|Kms|Meter|meter|mtrs|m)", distance_str)
        # match = re.search(r"(\d+(\.\d+)?)\s*(Km|KM|Meter|meter|km)", distance_str)
        if match:
            value = float(match.group(1))
            unit = match.group(3)
            if (unit.lower() == 'km') or (unit.lower() == 'kms'):
                return value * 1000
            elif (unit.lower() == 'meter') or (unit.lower() == 'm') or (unit.lower() == 'mtrs'):
                return value
        return None
    except Exception as e:
        logger.error("Error converting distance to meters: %s", e)
        return None

# Example usage
print(distance_to_meters("3.9Km"))  # Should return 3900.0
print(distance_to_meters("3.9 KM"))  # Should return 3900.0
print(distance_to_meters("500 ms"))  # Should return 500.0
print(distance_to_meters("Within 1.4km"))  # Should return None


3900.0
3900.0
500.0
1400.0


In [47]:
# Extract distances for each location
location_matrix = {}
for index, row in df.iterrows():
    distances = {}
    for location, distance in ast.literal_eval(row['LocationAdvantages']).items():
        print(distance,location)
        distances[location] = distance_to_meters(distance)
    location_matrix[index] = distances

# Convert the dictionary to a dataframe
location_df = pd.DataFrame.from_dict(location_matrix, orient='index')

# Display the first few rows
location_df.head()


800 Meter Bajghera Road
2.5 KM Palam Vihar Halt
3.1 KM DPSG Palam Vihar
3.1 KM Park Hospital
4.9 KM Gurgaon Railway Station
5.4 KM The NorthCap University
1.2 KM Dwarka Expy
7.7 KM Hyatt Place Gurgaon Udyog Vihar
7.2 KM Dwarka Sector 21, Metro Station
7.4 KM Pacific D21 Mall
14.7 KM Indira Gandhi International Airport
6.2 KM Hamoni Golf Camp
8.8 KM Fun N Food Waterpark
9 KM Accenture DDC5
1.4 Km DPSG Palam Vihar Gurugram
4.4 Km The NorthCap University
1.4 Km Park Hospital, Palam Vihar
8.2 Km Pacific D21 Mall
1.2 Km Palam Vihar Halt Railway Station
8.1 Km Dwarka Sector 21 Metro Station
450 m Dwarka Expressway
8.1 Km Fun N Food Water Park
14.1 Km Indira Gandhi International Airport
11.2 Km Tau DeviLal Sports Complex
5 Km Hamoni Golf Camp
6.1 Km Hyatt Place
11.2 Km Altrade Business Centre
2.7 Km AIPL Business Club Sector 62
2 Km Heritage Xperiential Learning School
2.5 Km CK Birla Hospital
3.5 Km Paras Trinity Mall Sector 63
3.8 Km Rapid Metro Station Sector 56
6.8 Km De Adventure Park
99

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
0,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,550.0,NaN,NaN,NaN,NaN,6700.0,3800.0,NaN,NaN,7500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,5300.0,NaN,NaN,NaN,2500.0,8800.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,1500.0,NaN,NaN,NaN,6500.0,6700.0,5100.0,NaN,NaN,8200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,5500.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
location_df.index = df.PropertyName
location_df.head()


,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M3M Crown,550.0,NaN,NaN,NaN,NaN,6700.0,3800.0,NaN,NaN,7500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adani Brahma Samsara Vilasa,5300.0,NaN,NaN,NaN,2500.0,8800.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sobha City,1500.0,NaN,NaN,NaN,6500.0,6700.0,5100.0,NaN,NaN,8200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Signature Global City 93,NaN,NaN,NaN,5500.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Replacing with big value because 0 would indicate that distance is too little and mis recommend
location_df.fillna(54000,inplace=True)
location_df


,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
M3M Crown,550.0,54000.0,54000.0,54000.0,54000.0,6700.0,3800.0,54000.0,54000.0,7500.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Adani Brahma Samsara Vilasa,5300.0,54000.0,54000.0,54000.0,2500.0,8800.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Sobha City,1500.0,54000.0,54000.0,54000.0,6500.0,6700.0,5100.0,54000.0,54000.0,8200.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Signature Global City 93,54000.0,54000.0,54000.0,5500.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Pyramid Urban Homes 2,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Satya The Hermitage,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0


In [40]:
from sklearn.preprocessing import StandardScaler
# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
location_df_normalized = pd.DataFrame(scaler.fit_transform(location_df), columns=location_df.columns, index=location_df.index)
location_df_normalized


,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-7.960979,-15.652476,-15.652476,-3.149592,-2.966108,-3.147217,-3.726615,-10.231739,-15.652476,-6.023233,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
M3M Crown,-7.998993,0.063888,0.063888,0.328277,0.368941,-3.054053,-3.529275,0.090308,0.063888,-6.009941,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Adani Brahma Samsara Vilasa,-7.276720,0.063888,0.063888,0.328277,-3.129124,-2.903557,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Sobha City,-7.854539,0.063888,0.063888,0.328277,-2.857430,-3.054053,-3.430606,0.090308,0.063888,-5.916893,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Signature Global City 93,0.128476,0.063888,0.063888,-2.985606,0.368941,0.335688,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,0.128476,0.063888,0.063888,0.328277,0.368941,0.335688,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Pyramid Urban Homes 2,0.128476,0.063888,0.063888,0.328277,0.368941,0.335688,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Satya The Hermitage,0.128476,0.063888,0.063888,0.328277,0.368941,0.335688,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888


In [42]:
cosine_sim3 = cosine_similarity(location_df_normalized)
cosine_sim3

array([[ 1.        ,  0.11228075,  0.08647048, ..., -0.01073953,
        -0.06042898, -0.06042898],
       [ 0.11228075,  1.        ,  0.13141512, ..., -0.00854903,
        -0.01763808, -0.01763808],
       [ 0.08647048,  0.13141512,  1.        , ..., -0.01142363,
        -0.02363121, -0.02363121],
       ...,
       [-0.01073953, -0.00854903, -0.01142363, ...,  1.        ,
         0.04377593,  0.04377593],
       [-0.06042898, -0.01763808, -0.02363121, ...,  0.04377593,
         1.        ,  1.        ],
       [-0.06042898, -0.01763808, -0.02363121, ...,  0.04377593,
         1.        ,  1.        ]])

In [43]:
def recommend_properties_with_scores(property_name, top_n=247):
    
    cosine_sim_matrix = 30*cosine_sim1 + 20*cosine_sim2 + 8*cosine_sim3
    # cosine_sim_matrix = cosine_sim3
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

In [44]:
# Test the recommender function using a property name
recommend_properties_with_scores('Ireo Victory Valley')

,PropertyName,SimilarityScore
0,Pioneer Urban Presidia,28.021460
1,Ambience Creacions,27.787913
2,DLF The Crest,24.205986
3,Pioneer Araya,23.415308
4,Silverglades The Melia,21.007840
...,...,...
240,JMS The Nation,-14.735656
241,Shree Vardhman City,-14.888274
242,JMS Prime Land,-15.082229
243,Vatika Aspiration,-15.099679
